In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install evaluate
!pip install seqeval

In [ ]:
%%capture
!pip install --upgrade transformers

In [ ]:
%%capture
!pip install matplotlib-venn
!pip install pydot
!pip install cartopy
import cartopy
import pydot

In [ ]:
%%capture
!pip install accelerate -U

In [ ]:
import torch
import pandas as pd
import os
import json
import re
import xml.etree.ElementTree as ET
from collections import Counter
from tqdm import tqdm
import nltk
import zipfile
import tarfile
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from google.colab import drive
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding
import json
import sqlite3
import csv
from tqdm import tqdm
import pandas as pd
import torch
from google.colab import drive
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding
import os
import pandas as pd
from lxml import etree
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ETree
from pathlib import Path
import glob
from tqdm import tqdm
from google.colab import drive
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda")
json_data = '/content/drive/MyDrive/davodi.json'
persian_df = pd.read_json(json_data)
persian_df = persian_df.drop(['_id', 'sentence', 'frame', 'lexicalUnit', 'status', 'issuer', 'is_active', 'createdAt', 'updatedAt', 'PId', 'lang', 'description', 'lexicalUnitHint', 'reviewer', 'lexicalUnitHelper', 'frameHelper', 'frameName', 'lexicalUnitName'], axis = 1)

In [ ]:
def framenet_tags(FN_tags):
    framenet_pattern = []
    for FN_tag in FN_tags:
        tag_type = FN_tag.get('tagType')
        if tag_type == 5:
            element = FN_tag.get('element', {})
            element_name = element.get('name', '')
            framenet_pattern.append(element_name)
        else:
            framenet_pattern.append('O')
    return framenet_pattern

In [ ]:
persian_df['frameNetTags'] = persian_df['frameNetTags'].apply(lambda FN_tags : framenet_tags(FN_tags))

In [ ]:
device = torch.device("cuda")
sentences_path = "/content/drive/MyDrive/sentences.zip"

In [ ]:
os.system("unzip sentences.zip")

2304

In [ ]:
zip_file_path = '/content/drive/My Drive/sentences.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/sentences')

In [ ]:
directory = "/content/drive/MyDrive/sentences"
dataframes = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filepath = os.path.join(directory, filename)
        enpb_df = pd.read_json(filepath, lines=True)
        dataframes.append(enpb_df)

In [ ]:
def read_json_files(root_dir):
  dataframes = []
  for root, dirs, files in os.walk(root_dir):
      for filename in files:
          if filename.endswith(".json"):
              filepath = os.path.join(root, filename)
              df = pd.read_json(filepath, lines=True)
              dataframes.append(df)
  return dataframes
dataframes = read_json_files("/content/drive/MyDrive/sentences")

In [ ]:
def prepare_token_classification_data(json_data):
    all_roles = []
    for key, value in json_data.items():
        predicate = value['predicate']
        roles = value['roles']
        verb_index = int(key)
        roles[verb_index] = "B-V"
        roles = ['O' if role == '_' else role for role in roles]
        all_roles.append(roles)
    return all_roles

In [ ]:
def create_token_label_dataframe_from_columns(df):
    sentence_ids = df["sentence_id"].tolist()
    words = df['words'].tolist()
    predictions = df['predictions'].tolist()
    df_prepared = pd.DataFrame(columns=["sentence_id", "words", "predictions"])
    for i, (sentence_id, word, prediction) in enumerate(zip(sentence_ids, words, predictions)):
        for label in prediction:
            df_prepared.loc[len(df_prepared)] = [sentence_id, word, label]
    return df_prepared

In [ ]:
enpb_df['predictions'] = enpb_df['predictions'].apply(lambda json_data : prepare_token_classification_data(json_data))

In [ ]:
enpb_df = create_token_label_dataframe_from_columns(enpb_df)

In [ ]:
enpb_df['Sentence'] = enpb_df['words'].apply(lambda x: ' '.join(x))

In [ ]:
enpb_df

,sentence_id,words,predictions,Sentence
0,0,"[And, without, more, ado, the, two, officers, ...","[O, AM-MNR, O, O, O, O, A0, B-V, O, A1, A2, O,...",And without more ado the two officers fastened...
1,0,"[And, without, more, ado, the, two, officers, ...","[AM-DIS, AM-MNR, O, O, O, O, A0, O, O, O, O, O...",And without more ado the two officers fastened...
2,1,"[She, would, fasten, the, ring, about, her, ne...","[A0, AM-MOD, B-V, O, A1, A2, O, O, O, O, O, O,...",She would fasten the ring about her neck and w...
3,1,"[She, would, fasten, the, ring, about, her, ne...","[A0, AM-MOD, O, O, O, O, O, O, O, B-V, A1, AM-...",She would fasten the ring about her neck and w...
4,1,"[She, would, fasten, the, ring, about, her, ne...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",She would fasten the ring about her neck and w...
...,...,...,...,...
26154,10260,"[It, was, after, the, first, snow, and, I, fol...","[O, O, O, O, O, O, O, A0, B-V, O, O, A1, AM-DI...",It was after the first snow and I followed the...
26155,10260,"[It, was, after, the, first, snow, and, I, fol...","[O, O, O, O, O, O, O, A0, O, O, O, O, O, O, O,...",It was after the first snow and I followed the...
26156,10260,"[It, was, after, the, first, snow, and, I, fol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",It was after the first snow and I followed the...
26157,10261,"[There, was, only, a, handful, of, mourners, a...","[O, B-V, O, O, A1, O, O, O, O, O, O, AM-CAU, O...",There was only a handful of mourners at Gillam...


In [ ]:
def process_label(label, all_words):
    start = label.get('start')
    end = label.get('end')
    if start is not None or end is not None:
        start = int(start)
        end = int(end)
        accumulated_chars = 0
        start_word_index = 0
        end_word_index = 0
        for i, word in enumerate(all_words):
            word_length = len(word)
            if accumulated_chars <= start:
                start_word_index = i
            if accumulated_chars >= end:
                end_word_index = i - 1
                break
            accumulated_chars += word_length + 1
        if end_word_index is None:
            end_word_index = len(all_words) - 1
        extracted_word = ' '.join(all_words[start_word_index:end_word_index + 1])
        return start, end, label.get('name'), extracted_word
    else:
        return None

In [ ]:
def annotate_sentence(df):
    sentence = df['Sentence']
    words = df['words']
    start = df['Start']
    end = df['End']
    annotation = df['Annotated Names']
    annotated_words = df['Extracted Words']
    result = []
    char_index = 0
    for word in words:
        label = 'O'
        word_start = char_index
        word_end = char_index + len(word) - 1
        if word in annotated_words.split():
            label = annotation
        result.append(label)
        char_index = word_end + 1
    return result

In [ ]:
xml_folder_path = '/content/drive/My Drive/framenetdata-1.7/fndata-1.7/lu'
xml_files = glob.glob(xml_folder_path + '/*.xml')
table = []
namespace = {'fn': 'http://framenet.icsi.berkeley.edu'}

In [ ]:
for filename in tqdm(xml_files):
    try:
        tree = ET.parse(filename)
        root = tree.getroot()
        for sentence in root.findall('.//fn:sentence', namespace):
            text_element = sentence.find('fn:text', namespace)
            if text_element is not None and text_element.text:
                all_words = text_element.text.split()
                annotation_sets = sentence.findall('.//fn:annotationSet[@status="MANUAL"]', namespace)
                for annotation_set in annotation_sets:
                    layers = annotation_set.findall('.//fn:layer[@name="FE"]', namespace)
                    for layer in layers:
                        labels = layer.findall('.//fn:label', namespace)
                        for label in labels:
                            processed_label = process_label(label, all_words)
                            if processed_label is not None:
                                table.append((text_element.text,) + processed_label)
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        print(f"XML file Name:" + str(filename))

 41%|████      | 1007/2483 [01:19<00:35, 42.12it/s]

Error parsing XML file: no element found: line 2, column 0
XML file Name:/content/drive/My Drive/framenetdata-1.7/fndata-1.7/lu/lu3973.xml


100%|██████████| 2483/2483 [01:37<00:00, 25.54it/s] 


In [ ]:
enfn_df = pd.DataFrame(table, columns=['Sentence', 'Start', 'End', 'Annotated Names', 'Extracted Words'])
enfn_df["words"] = enfn_df["Sentence"].apply(lambda sentence: [word.strip() for word in sentence.split(' ')])
enfn_df['frameNetTags'] = enfn_df.apply(annotate_sentence, axis=1)
enfn_df

,Sentence,Start,End,Annotated Names,Extracted Words,words,frameNetTags
0,` What makes you think Rainbow would entertain...,76,81,Entity_2,of you,"[`, What, makes, you, think, Rainbow, would, e...","[O, O, O, Entity_2, O, O, O, O, O, O, O, O, O,..."
1,` What makes you think Rainbow would entertain...,70,74,Entity_1,likes,"[`, What, makes, you, think, Rainbow, would, e...","[O, O, O, O, O, O, O, O, O, O, O, O, O, Entity..."
2,It aims to compete with the likes of BMW and A...,34,48,Entity_2,of BMW and Audi,"[It, aims, to, compete, with, the, likes, of, ...","[O, O, O, O, O, O, O, Entity_2, Entity_2, Enti..."
3,It aims to compete with the likes of BMW and A...,28,32,Entity_1,likes,"[It, aims, to, compete, with, the, likes, of, ...","[O, O, O, O, O, O, Entity_1, O, O, O, O, O, O,..."
4,They anathematised and expelled the likes of A...,42,89,Entity_2,"of Arius , who taught that Jesus was fully human","[They, anathematised, and, expelled, the, like...","[O, O, O, O, O, O, Entity_2, Entity_2, Entity_..."
...,...,...,...,...,...,...,...
59142,"During his four days in Iraq , Hall said he wa...",145,169,Undesirable_event,from chronic malnutrition,"[During, his, four, days, in, Iraq, ,, Hall, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, Und..."
59143,"During his four days in Iraq , Hall said he wa...",100,126,Patient,a quarter of Iraqi children,"[During, his, four, days, in, Iraq, ,, Hall, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
59144,"But the many whose lives he had ruined , whose...",80,111,Undesirable_event,torture and death at his hands,"[But, the, many, whose, lives, he, had, ruined...","[O, O, O, O, O, O, O, O, O, O, O, Undesirable_..."
59145,"But the many whose lives he had ruined , whose...",47,65,Patient,nearest and dearest,"[But, the, many, whose, lives, he, had, ruined...","[O, O, O, O, O, O, O, O, O, O, Patient, Patien..."


In [ ]:
endf = pd.merge(enpb_df, enfn_df, on='Sentence', how='inner')

In [ ]:
persian_df = persian_df.rename(columns={'words': 'Sentence'})
df_merged = pd.DataFrame()
for i in range(len(persian_df)):
    df_merged = pd.concat([df_merged, persian_df.iloc[[i]], endf.iloc[[i]]], ignore_index=True)

In [ ]:
train_data, temp_data = train_test_split(endf, test_size=0.2, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))
train_data.to_csv("train_set.csv", index=False)
validation_data.to_csv("validation_set.csv", index=False)
test_data.to_csv("test_set.csv", index=False)

Training set size: 80
Validation set size: 10
Test set size: 10


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
tokenized_input = tokenizer(endf['Sentence'][0].split(' '), is_split_into_words=True)
tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

['[CLS]',
 'he',
 'seemed',
 'a',
 'pleasant',
 'well',
 'spoken',
 'quiet',
 'young',
 'man',
 '.',
 '[SEP]']

In [ ]:
endf = endf.rename(columns={'predictions' : 'propBankTags'})

In [ ]:
def create_role_id_dictionary():
    """Creates a dictionary that maps PropBank roles to numerical IDs."""
    roles = set()
    frame_elements = set()
    for _, row in endf.iterrows():
        prop_str = row['propBankTags']
        framenet_str = row['frameNetTags']
        roles.update(prop_str)
        frame_elements.update(framenet_str)
    role_id_dict = defaultdict(int)
    frame_id_dict = defaultdict(int)
    index = 1
    for role in roles:
        role_id_dict[role] = index
        index += 1
    for frame_element in frame_elements:
        frame_id_dict[frame_element] = index
        index += 1
    return role_id_dict, frame_id_dict, roles, frame_elements

In [ ]:
role2id, frame2id, roles, frame_elements = create_role_id_dictionary()
print("Role2ID:" + str(role2id))
print("Frame2ID:" + str(frame2id))
print("Roles:" + str(roles))
print("Frame Elements:" + str(frame_elements))

Role2ID:defaultdict(<class 'int'>, {'AM-TMP': 1, 'AM-PNC': 2, 'O': 3, 'AM-NEG': 4, 'AM-MNR': 5, 'C-A1': 6, 'A1': 7, 'A2': 8, 'B-V': 9, 'R-A1': 10, 'AM-ADV': 11, 'AM-DIS': 12, 'AM-LOC': 13, 'R-A2': 14, 'AM-DIR': 15, 'A0': 16, 'AM-MOD': 17})
Frame2ID:defaultdict(<class 'int'>, {'Cause': 18, 'Means': 19, 'Cognizer': 20, 'O': 21, 'Descriptor': 22, 'Agent': 23, 'Group': 24, 'New_member': 25, 'Entity': 26, 'Category': 27, 'Particular_iteration': 28, 'Age': 29, 'Manner': 30, 'Item': 31})
Roles:{'AM-TMP', 'AM-PNC', 'O', 'AM-NEG', 'AM-MNR', 'C-A1', 'A1', 'A2', 'B-V', 'R-A1', 'AM-ADV', 'AM-DIS', 'AM-LOC', 'R-A2', 'AM-DIR', 'A0', 'AM-MOD'}
Frame Elements:{'Cause', 'Means', 'Cognizer', 'O', 'Descriptor', 'Agent', 'Group', 'New_member', 'Entity', 'Category', 'Particular_iteration', 'Age', 'Manner', 'Item'}


In [ ]:
role2id, frame2id, roles, frame_elements = create_role_id_dictionary()

In [ ]:
endf['propBankTags'] = endf['propBankTags'].apply(lambda label : [role2id[l] for l in label])

In [ ]:
def tokenize_and_align_labels(examples):
    #tokenized_inputs = tokenizer(examples["Sentence"].split(' '), truncation=True, padding=True, is_split_into_words=True)
    #tokenized_inputs = tokenizer(examples["Sentence"], truncation=True, padding=True, is_split_into_words=True)
    tokenized_inputs = tokenizer(examples["words_x"], truncation=True, padding=True, is_split_into_words=True)#words_x
    #tokenized_inputs = tokenizer(examples["Words"], truncation=True, padding=True, is_split_into_words=False)
    labels = []
    for i, label in enumerate(examples[f"propBankTags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(endf)
tokenized_data = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_data = tokenized_data.train_test_split(test_size=0.3)

In [ ]:
import evaluate
seqeval = evaluate.load("seqeval")

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=len(role2id)+1,ignore_mismatched_sizes=True)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label2id = dict(role2id)

In [ ]:
id2label = dict((v,k) for k,v in label2id.items())

In [ ]:
label_list = list(role2id.keys())
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import classification_report
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels_bin = list(np.concatenate(true_labels))
    pred_labels_bin = list(np.concatenate(true_predictions))
    print(true_labels_bin)
    print(pred_labels_bin)
    report = classification_report(true_labels_bin, pred_labels_bin)
    metrics = precision_recall_fscore_support(true_labels_bin, pred_labels_bin, average='weighted')
    accuracy = metrics[0]
    precision = metrics[1]
    recall = metrics[2]
    f1_score = metrics[3]
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1_score,
        "accuracy": accuracy
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="bert-token-classification",
    learning_rate=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    weight_decay=0.01,
    save_strategy="no",
    report_to=None,
    #use_cpu=True,
    #no_cuda=True,
    #load_best_model_at_end=True, # uncomment if you wanna save
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=tokenized_data['train']  ,eval_dataset=tokenized_data['test'],compute_metrics=compute_metrics,)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()